# Column in Simple Construction

Subtitle

## Setup

### Imports & Env. Settings

In [1]:
# Imports
import handcalcs # https://github.com/connorferster/handcalcs/wiki
from handcalcs import render # https://github.com/connorferster/handcalcs?tab=readme-ov-file#basic-usage-1-as-a-jupyter-cell-magic-render
import forallpeople as si # https://connorferster.github.io/forallpeople/basic_usage.html
from modules.section import UniversalColumn
from modules.strength import design_strength

In [2]:
# Environment config.
# --> SI units structural env ( https://connorferster.github.io/forallpeople/environments.html#Environments )
si.environment("structural", top_level=True) # load SI units for structural eng. into the environment, top_level=True defines them in the top-level namespace (as vars)
# -> Handcalcs environment settings for TeX rendering
handcalcs.set_option("latex_block_start", "$")
handcalcs.set_option("latex_block_end", "$")
handcalcs.set_option("math_environment_start", "aligned")
handcalcs.set_option("math_environment_end", "aligned")

### Question Data

In [26]:
%%render params 2 # Reactions from beams, and length
# Actions
# F_c = 1 * kN
R_A = 200 * kN
R_B = 75 * kN
F_swt = 5 * kN

# Dimensions
L = 7 * m



<IPython.core.display.Latex object>

### Trial Section

In [4]:
trial_section = UniversalColumn.create(designation="203x203x52")
# ... UB not implemented, should be in rework of sections module

In [5]:
%%render params 1 # BS Notation
D = trial_section.D * mm
B = trial_section.B * mm
d = trial_section.d * mm
t = trial_section.t * mm
T = trial_section.T * mm

S_xx = trial_section.S_xx * 1000*mm**3
S_yy = trial_section.S_yy * 1000*mm**3
Z_xx = trial_section.Z_xx * 1000*mm**3
Z_yy = trial_section.Z_yy * 1000*mm**3
r_xx = trial_section.r_xx * 10*mm
r_yy = trial_section.r_yy * 10*mm
A = trial_section.A * 100*mm**2

<IPython.core.display.Latex object>

### Loading and Moments

In [6]:
%%render short
F = R_A + R_B + F_swt
e_x = D/2 + 100 * mm
e_y = t/2 + 100 * mm
M_x = R_A * e_x
M_y = R_B * e_y


<IPython.core.display.Latex object>

<!-- ### Classification -->

### Effective Length $L_{\text{E}}$ (and $L_{\text{Cr}}$)

Table 22 BS 5950
_ENV 1993-1-1:2004_

In [7]:
%%render short 2 # Uncomment correct value, defaults to 1.0 L
# L_E = 0.7 * L
L_E = 0.85 * L # 📝
# L_E = 1.0 * L
# L_E = 1.2 * L
# L_E = 1.5 * L
# L_E = 2.0 * L

L_Cr = L_E

<IPython.core.display.Latex object>

## BS 5950

### Classification

In [8]:
%%render short 2
T
p_y = 265 * MPa # 📝

<IPython.core.display.Latex object>

In [9]:
# Section Class Table 11 BS

In [10]:
%%render short 2
lamb_x = L_E / r_xx
lamb_y = L_E / r_yy

lamb_c = min(lamb_x, lamb_y) 
p_c = 103 * MPa # 📝

<IPython.core.display.Latex object>

In [11]:
%%render short 2
lamb_LT = 0.5 * L / r_yy 

p_b = 193 * MPa # 📝
M_bs = p_b * S_xx

<IPython.core.display.Latex object>

### Stability Check

In [12]:
%%render short 2
P_c = p_c * A
util_st = F/P_c + M_x/M_bs + M_y/(p_y * Z_yy)


<IPython.core.display.Latex object>

## Eurocode 3

In [13]:
%%render params 1 # EC3 Notation
designation = trial_section.designation
h = trial_section.D * mm
b = trial_section.B * mm
t_w = trial_section.t * mm
t_f = trial_section.T * mm

W_ply = S_xx
W_plz = S_yy
W_ely = Z_xx
W_elz = Z_yy
i_yy = r_xx
i_zz = r_yy
A = trial_section.A * 100*mm**2

<IPython.core.display.Latex object>

In [14]:
%%render short 3
N_Ed = R_A + R_B + F_swt
e_y = h / 2 + 100*mm
e_z = t_w / 2 + 100*mm
M_y = R_A * e_y
M_z = R_B * e_z

<IPython.core.display.Latex object>

In [15]:
%%render short 3
t_f 
f_y = p_y
epsilon = (235*MPa/f_y)**0.5

<IPython.core.display.Latex object>

In [16]:
%%render params 2
pi = 3.14159265359
E = 210 * GPa
G = 81 * GPa
gamma_m_0 = 1.0
gamma_m_1 = 1.0


<IPython.core.display.Latex object>

In [17]:
%%render short 3
lamb_1 = pi * (E/f_y)**0.5
L_Crz = L_E
lamb_z = L_Crz / i_zz
lamb_prime_z = lamb_z/lamb_1 
hoverb = h/b

<IPython.core.display.Latex object>

In [18]:
%%render short 3
# read alpha from table
alpha = 0.49 # 📝
Phi = 0.5 * (1 + alpha * (lamb_prime_z - 0.2) +  lamb_prime_z**2)
chi_z =  (Phi + (Phi**2 - lamb_prime_z**2)**0.5)**-1
N_bzRd = chi_z * A * f_y/gamma_m_1


<IPython.core.display.Latex object>

In [19]:
%%render params 2
I_zz = trial_section.I_yy * 10000*mm**4
L_cr = 5.95 * m # 📝
I_w = trial_section.H *1000000 * mm**6
I_T = trial_section.J * 10000 *mm**4

f_y = 275 * MPa



<IPython.core.display.Latex object>

M_cr

In [20]:
%%render long 2
M_cr = ( (pi**2 * E * I_zz)/L_cr**2 ) * ( ((I_w)/(I_zz)) + ((G * I_T* L_cr**2)/(E * I_zz * pi**2)) )**0.5

<IPython.core.display.Latex object>

In [21]:
%%render short 2
lamb_LT = (W_ply * f_y / M_cr)**0.5


<IPython.core.display.Latex object>

In [22]:
%%render short 3
# read alpha_LT from table
alpha_LT = 0.34 # 📝
beta = 0.7
lamb_LT_0 = 0.4
Phi_LT = 0.5 * (1 + alpha_LT * (lamb_LT - lamb_LT_0) + beta * lamb_LT**2)
chi_LT =  (Phi_LT + (Phi_LT**2 - beta*lamb_LT**2)**0.5)**-1
chi_LT_limit = 1 / lamb_LT



<IPython.core.display.Latex object>

In [23]:
%%render 3
M_bRd = chi_LT * W_ply * f_y / gamma_m_1


<IPython.core.display.Latex object>

In [24]:
%%render short 3
M_zRd = W_elz * f_y / gamma_m_1

<IPython.core.display.Latex object>

In [25]:
%%render short 3
util_int = N_Ed/N_bzRd + M_y/M_bRd + 1.5*(M_z/M_zRd)

<IPython.core.display.Latex object>